In [47]:
# from preloaded import open
import pandas as pd
import numpy as np

In [2]:
gamemap = """
? ? ? ? ? ?
? ? ? ? ? ?
? ? ? 0 ? ?
? ? ? ? ? ?
? ? ? ? ? ?
0 0 0 ? ? ?
""".strip()

result = """
1 x 1 1 x 1
2 2 2 1 2 2
2 x 2 0 1 x
2 x 2 1 2 2
1 1 1 1 x 1
0 0 0 1 1 1
""".strip()

n = 6

In [3]:
def open(y, x):
    df = to_df(result)
    res = df.iloc[y,x]
    if res == 'x':
        raise ValueError(f'Oooopsyy, stepped on {res}, coords:({y},{x})')
    return df.iloc[y,x]

In [ ]:
def to_arr(x):
    if isinstance(x, str):
        return np.array([i.split(' ') for i in x.split('\n')])
        
    else:
        raise TypeError(f'x({x}) is {type(x)}, not str')

def from_arr(x):
    if isinstance(x, np.ndarray):
        return np.array2string(x).replace('[', '').replace(']', '').replace('\n ', '\n')
    else:
        raise TypeError(f'x({x}) is {type(x)}, not numpy.ndarray')

In [65]:
np.array2string(np.array([[1,2],[3,4]])).replace('[', '').replace(']', '').replace('\n ', '\n')

'1 2\n3 4'

In [42]:
def to_df(x):
    if isinstance(x, str):
        return pd.DataFrame([i.split(' ') for i in x.split('\n')]).astype(str)
    else:
        raise TypeError(f'x({x}) is {type(x)}, not str')

In [43]:
def from_df(x):
    if isinstance(x, pd.DataFrame):
        return x.to_string(header=False, index=False)
    else:
        raise TypeError(f'x({x}) is {type(x)}, not DataFrame')

In [6]:
def cells_near(y, x, shape):
    y_max, x_max = shape
    res = []
    for x_s in range(x-1, x+2):
        for y_s in range(y-1, y+2):
            if x_s >= 0 and y_s >= 0 and x_s < x_max and y_s < y_max and (x != x_s or y != y_s):
                res.append((y_s, x_s))
    return tuple(res)

In [32]:
def bf_checker(gmap, n):
    
    df = to_df(gmap)
    y_max, x_max = df.shape
    
    # Just check the matrix for errors
    for y in range(y_max):
        for x in range(x_max):
            # Getting coords of all cells near current
            c_n = cells_near(y, x, df.shape)

            if df.iloc[y,x] not in ['?', 'x']:
                bombs = int(df.iloc[y,x])    # quantity of bombs around
                quests = []                  # list for '?' coords
                find_bombs = 0               # number for bombs already found around

                # Gets amount of 'x' around
                for yn, xn in c_n:
                    if df.iloc[yn, xn] == 'x':
                        find_bombs += 1

                # If amount of 'x' around not equal the cell value, retruns errored matrix
                if find_bombs != bombs:
                    return from_df(df), -1
                
    return from_df(df), 0
        

In [33]:
def cycle_1(gmap, n):
    print('\n','CYCLE 1','\n')
    df = to_df(gmap)
    y_max, x_max = df.shape
    bombs_count = 0
    
    old_df = pd.DataFrame([None])
        
    while old_df.values.tolist() != df.values.tolist():
        
        old_df = df.copy()
        
        print(from_df(df), '\n', sep='\n')
        # Open clean matrix
        for y in range(y_max):
            for x in range(x_max):
                
                # Getting coords of all cells near current
                c_n = cells_near(y, x, df.shape)
                
                # If cell value greater than 0 and not '?' or 'x'
                if df.iloc[y,x] not in ['?', 'x']:
                    
                    bombs = int(df.iloc[y,x])    # quantity of bombs around
                    quests = []                  # list for '?' coords
                    find_bombs = 0               # number for bombs already found around

                    # Gets the amount and locations of '?'
                    for yn, xn in c_n:
                        if df.iloc[yn, xn] == '?':
                            quests.append((yn,xn))
                        if df.iloc[yn, xn] == 'x':
                            find_bombs += 1

                    # if bombs equal quantity of '?', changes all '?' to 'x'
                    if bombs - find_bombs == len(quests):
                        for yn, xn in quests:
                            df.iloc[yn,xn] = 'x'
                            bombs_count += 1
                            
                    # if all bombs already found, opens all '?'
                    elif find_bombs == bombs:
                        for yn, xn in quests:
                            # print(from_df(df), f'coords: {y, x} quest: {yn, xn}', '\n', sep='\n')
                            df.iloc[yn,xn] = str(open(yn, xn))
                            
                    # else:
                    #     print(from_df(df), f'coords: {y, x}, quests: {quests}, bombs: {bombs}, find_bombs: {find_bombs}', sep='\n')
                    #     raise ValueError('Unexpected condition')
    
    return from_df(df), n - bombs_count

In [35]:
def cycle_2(gmap, n):
    print('\n','CYCLE 2','\n')
    df = to_df(gmap)
    y_max, x_max = df.shape
    quests = []
    for y in range(y_max):
        for x in range(x_max):
            if df.iloc[y, x] == '?':
                quests.append((y,x))

    alphabet = 'x?'
    combos = []

    for current in range(len(quests)):
        a = [i for i in alphabet]

        for y in range(current):
            a = [x+i for i in alphabet for x in a]
        
        for i in a:
            if i.count('?') == len(quests) - n and i.count('x') == n:
                combos.append(i)
    print('kek')
    solves = []
    for combo in combos:
        bf_df = df.copy()
        n = 0
        for i in range(len(quests)):
            bf_df.iloc[quests[i]] = combo[i]
            
        bf_gmap, n = bf_checker(from_df(bf_df), n)
        if n == 0:
            solves.append(bf_gmap)
            print('solves', *solves, sep='\n')
        if len(solves) > 1:
            return '?', -1
        
    # print('solves', *solves, sep='\n')
    if len(solves) == 1:
        res, n = cycle_1(solves[0], 0)
        return res, n
        
    else:
        return '?', -1

In [10]:
def solve_mine(gmap, n):
    
    gmap, n = cycle_1(gmap, n)
    
    if gmap.count('?') or n != 0:
        gmap, n = cycle_2(gmap, n)
    
    if gmap.count('?') == 0 and n == 0:
        return gmap
                    
    else:
        return '?'

In [44]:
res = solve_mine(gamemap, n)
if res == result:
    print('HuH')


 CYCLE 1 

? ? ? ? ? ?
? ? ? ? ? ?
? ? ? 0 ? ?
? ? ? ? ? ?
? ? ? ? ? ?
0 0 0 ? ? ?


? ? ? ? ? ?
? ? 2 1 2 ?
? ? 2 0 1 ?
? ? 2 1 2 ?
1 1 1 1 ? ?
0 0 0 1 ? ?


? ? ? ? ? ?
? ? 2 1 2 ?
? x 2 0 1 ?
2 x 2 1 2 ?
1 1 1 1 x 1
0 0 0 1 1 1


? ? ? ? ? ?
? 2 2 1 2 ?
2 x 2 0 1 ?
2 x 2 1 2 2
1 1 1 1 x 1
0 0 0 1 1 1


? ? ? ? ? ?
2 2 2 1 2 ?
2 x 2 0 1 x
2 x 2 1 2 2
1 1 1 1 x 1
0 0 0 1 1 1


? ? ? ? ? ?
2 2 2 1 2 2
2 x 2 0 1 x
2 x 2 1 2 2
1 1 1 1 x 1
0 0 0 1 1 1



 CYCLE 2 

kek
solves
? x ? ? x ?
2 2 2 1 2 2
2 x 2 0 1 x
2 x 2 1 2 2
1 1 1 1 x 1
0 0 0 1 1 1

 CYCLE 1 

? x ? ? x ?
2 2 2 1 2 2
2 x 2 0 1 x
2 x 2 1 2 2
1 1 1 1 x 1
0 0 0 1 1 1


1 x 1 1 x 1
2 2 2 1 2 2
2 x 2 0 1 x
2 x 2 1 2 2
1 1 1 1 x 1
0 0 0 1 1 1


HuH
